In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

%matplotlib inline

In [2]:
postulations_encoded = pd.read_csv('../MapDataAnalysis/MappedData4M.csv')

In [3]:
postulations_encoded.head()

,Unnamed: 0,descripcion,idaviso,idpostulante,nombre_area,nombre_sort,se_postulo,titulo,estadoEncoded,tipoDeTrabajoEncoded,nivelLaboralEncoded,sexoEncoded,edadEncoded,zonaEncoded
0,0,0,1111662226,EzpVOLz,Farmacéutica,3.0,1,Cajeros/as- Atencion al cliente,0,0,2,1,3.0,0
1,1,0,1111413600,kPjordV,Telemarketing,3.0,1,Operador telefónico / Retención y Fidelizació...,0,1,2,1,0.0,1
2,2,1,1112401116,pzNk9Rr,Soporte Técnico,1.0,1,Técnico en Reparación de Celulares,1,0,1,1,3.0,0
3,3,0,1112319004,vV3BMz1,Ventas,1.0,1,Asesores Comerciales NEUQUEN,1,0,1,1,4.0,1
4,4,0,1112094756,a4GRrN,Call Center,3.0,1,Representante de Atencion al Cliente/ Banco,2,1,0,1,3.0,0


In [4]:
postulations_encoded= postulations_encoded.drop(['idaviso','idpostulante','Unnamed: 0', 'titulo','estadoEncoded','sexoEncoded','zonaEncoded','nombre_area','nivelLaboralEncoded','descripcion'],axis=1)

In [5]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column in postulations_encoded.columns:
    if postulations_encoded[column].dtype == type(object):
        postulations_encoded[column] = le.fit_transform(postulations_encoded[column].astype(str))

postulations_encoded.sample(5)

,nombre_sort,se_postulo,tipoDeTrabajoEncoded,edadEncoded
1518635,3.0,1,0,2.0
609939,1.0,1,0,2.0
1291854,1.0,1,0,2.0
2275024,3.0,0,0,0.0
2385409,2.0,0,0,3.0


In [6]:
#Normalizamos los valores
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(postulations_encoded) 
postulations_encoded.loc[:,:] = scaled_values

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(postulations_encoded.drop(['se_postulo'], axis=1).values, postulations_encoded['se_postulo'], test_size=0.30, random_state=75)

In [8]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.5,random_state=123).fit(X_train, y_train)

In [9]:
clf.predict(X_test)

array([0., 0., 0., ..., 1., 0., 1.])

In [10]:
clf.score(X_test,y_test)

0.5595371463498363

In [11]:
test = pd.read_csv('../MapDataAnalysis/MappedTest.csv')

In [12]:
test.head()

,id,idaviso,idpostulante,nombre_sort,titulo,descripcion,nombre_area,estadoEncoded,tipoDeTrabajoEncoded,nivelLaboralEncoded,sexoEncoded,edadEncoded,zonaEncoded
0,0,739260,6M9ZQR,6.0,Asistente Comercial,1,Comercial,1,0,3,0,4.0,1.0
1,1,739260,6v1xdL,2.0,Asistente Comercial,1,Comercial,1,0,3,1,3.0,1.0
2,2,739260,ezRKm9,3.0,Asistente Comercial,1,Comercial,1,0,3,0,3.0,1.0
3,3,758580,1Q35ej,4.0,Oracle Financials Sr. Application Developer,0,Tecnologia / Sistemas,1,0,2,1,4.0,1.0
4,4,758580,EAN4J6,3.0,Oracle Financials Sr. Application Developer,0,Tecnologia / Sistemas,2,0,2,0,3.0,1.0


In [13]:
test = test.drop(['idaviso','idpostulante','titulo','estadoEncoded','sexoEncoded','zonaEncoded','nombre_area','nivelLaboralEncoded','descripcion'],axis=1)

In [14]:
le = preprocessing.LabelEncoder()
for column in test.columns:
    if test[column].dtype == type(object):
        test[column] = le.fit_transform(test[column].astype(str))

In [15]:
#Normalizamos los valores
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(test.drop(columns = ['id'])) 
test.loc[:,1:] = scaled_values

In [16]:
test.head()

,id,nombre_sort,tipoDeTrabajoEncoded,edadEncoded
0,0,1.000000,0.0,1.00
1,1,0.333333,0.0,0.75
2,2,0.500000,0.0,0.75
3,3,0.666667,0.0,1.00
4,4,0.500000,0.0,0.75


In [17]:
result = clf.predict_proba(test.drop(['id'],axis = 1))

In [18]:
result2 = []
for elem in result:
    result2.append(elem[1])

In [19]:
test['sepostulo'] = result2

In [20]:
test = test[['id','sepostulo']]

In [21]:
test.head()

,id,sepostulo
0,0,0.397418
1,1,0.439069
2,2,0.483540
3,3,0.356534
4,4,0.483540


In [22]:
test.to_csv('../Predictions/GradientBoosting4MLessFeatures1.csv', index=False)